# Part 1 - Word counting
---------------

For the first part of the practical assignment, we will use MapReduce paradigms to implement a word counting program in Apache Spark. First, some setup and data preparation. For further information on the software, check out the documentation [here](https://spark.apache.org/docs/latest/api/python/pyspark.html).

To run the code in a cell, select it and press Ctrl+Enter, or use the `Run` button at the top of the notebook.

In [ ]:
import pyspark

sc = pyspark.SparkContext("local[*]", "PUC Big Data workshop")

We load the text file containing Shakespeare's famous Romeo & Juliet using a convenient PySpark function. This will automatically split the text file into separate lines.

In [ ]:
lines = sc.textFile("shakespeare.txt")

To give you an idea of the contents of this file, we will print a few random lines. The `takeSample` operation can be very useful to examine the data you are working with; particularly when working with a large amount of data, it is unfeasible to look at it all, so looking at a random small subset can give you a rough idea of what you are working with. Feel free to sample the results of the individual cells below to see what is happening!

In [ ]:
lines.takeSample(withReplacement=False, num=10)

Since some lines in the file are empty, we first filter those out; they do not contain words, so we don't need them! The `filter` operation will give us all lines with a length larger than 0.

In [ ]:
non_empty_lines = lines.filter(lambda line: len(line) > 0)

We will extensively use `lambda` functions when working with Spark, since they allow for a nice and short notation of simple operations. Usually, the function will take one argument and perform some action on that argument. For example, the function we just used takes a single line, and checks whether its length is larger than 0 (e.g. it is not empty).

Now that we have a list of non-empty sentences, we will split these sentences into single words. Since this is just an exercise, we will simply split the sentences on space characters; this will not give a perfect split, but it is good enough to use for the rest of the program. `flatMap` will make sure that we don't end up with nested lists and instead just give us one long list of words.

In [ ]:
words = non_empty_lines.flatMap(lambda line: line.strip().split())

The `strip` function removes leading and trailing spaces from the line, and the `split` function splits the line on spaces.

Let's look at some of the words we ended up with!

In [ ]:
words.takeSample(withReplacement=False, num=10)

We loaded the text of Shakespeare's famous Romeo & Juliet, removed empty lines and split the remaining lines on spaces using the `flatMap` function. Displayed above are 25 random words sampled from the split text. As you can see, splitting the sentences on spaces does not result in a perfect separation of words but it will do for our purposes.

------------------------

Now, let's implement a simple word count! First, we will use the `map` operation to transform each word into a (word, 1) tuple as per the slides.

In [ ]:
# Replace the dots with the right code!

annotated_words = words.map(lambda word: ...)       

After annotating each word with the number 1, we can then perform the shuffle step by using the `groupByKey` operation. This will group all (word, 1) tuples with identical keys (words) to the same worker node, and transform the (word, 1) tuples into (word, [1, 1, ...]) tuples, with all the 1's for a single word grouped into a list.

In [ ]:
grouped_words = annotated_words.groupByKey()

To obtain the total word counts, we simply need to add up all the 1's for each word. We can do that by using the `mapValues` operation. This will pass all the 1's in each (word, [counts]) tuple into the sum function, so we end up with (word, totalCount) tuples.

In [ ]:
# Replace the dots with the right code!
# Hint: look at the builtin Python sum function:
# https://docs.python.org/3/library/functions.html#sum

word_counts = grouped_words.mapValues(...)

Now, in order to make the next steps easier, we first swap the positions of the words and their counts in the tuples so we end up with (count, word) instead of (word, count). This makes the count the key of the item, and will allow us to sort by key to see which words are most common. We need to do this because Spark does not allow us to sort by values, only by keys.

To access the individual values in a tuple, you can use the bracket syntax:
    
```python
>>> x = ("hello", 1)
>>> x[0]
"hello"
>>> x[1]
1
>>> (x[1], x[0])
(1, "hello")
```

In [ ]:
# Replace the dots with the right code!

word_counts_reordered = word_counts.map(lambda x: ...)

Now we can sort the tuples by their counts in descending order, putting the most frequent words at the beginning of the list.

In [ ]:
sorted_word_counts = word_counts_reordered.sortByKey(ascending=False)

That's it! Let's look at the top 10 most frequent words in Romeo & Juliet.

In [ ]:
for (count, word) in sorted_word_counts.take(10):
    # print the word following by its count
    print(f"{word:<4}: {count:<6} occurrences")

### Well done!
That's it for the first part of this assignment. After the next part of the lecture, we can continue with part 2!

# Part 2 - Averaging player scores
---
In this part of the assignment, we will tackle a slightly more complicated problem; averaging the recorded scores of all players of a game. Since we do not actually have such a dataset, we will first create one ourselves!

First, let's load the names. We use the `takeSample` method here to enlarge the list of names times 10, in order to simulate multiple player scores.

In [ ]:
with open("names.txt") as f:
    names = sc.parallelize(f.read().split())

# Sample ten times the original number of names
names = sc.parallelize(names.takeSample(withReplacement=True, num=names.count() * 10))

names.takeSample(withReplacement=False, num=10)

Now we can generate some scores for the names! We will use `numpy` to generate the scores, and use Spark's `map` operation to assign them to the names.

In [ ]:
import numpy

# Generate as many scores as we have names in the `names` list
scores = numpy.random.randint(100, 10000, size=names.count())

names_and_scores = names.zip(sc.parallelize(scores))
names_and_scores.takeSample(withReplacement=False, num=10)

As you can see, we now have (name, score) tuples in the `names_and_scores` list. Now, let's think about how to calculate the average score over all players. Intuitively, we could use a simple strategy:

1. Calculate the average score per play using `groupByKey` and `mapValues`.
2. Calculate the average total score from all the individual averages.

First, we will calculate the actual expected average score in order to later check if the result of our strategy is correct.

In [ ]:
numpy.mean(scores)

Now we will implement our first strategy! Fill in the missing code in place of the dots in the cell below.

In [ ]:
# Replace the dots with the right code!

grouped_names_and_scores = names_and_scores.groupByKey()
# Hint: the average is sum / length!
names_with_average_scores = grouped_names_and_scores.mapValues(lambda x: ...)
# Hint: to extract the score from the tuple, use x[1].
# To get the number of values in a list, use list.count()
average_score = names_with_average_scores.map(lambda x: ...).sum() / ...

average_score

The average turns out to be different! Let's think about why that is. Imagine a smaller example:

```python
data = [('a', [1]), ('b', [2, 3]), ('c', [4, 5, 6])]
```

The average that we would like to calculate is `(1 + 2 + 3 + 4 + 5 + 6) / 6 = 3.5`. However, if we translate the strategy we used to a calculation, we end up with the following:

```python
mean_a = 1 / 2            # equals 0.5
mean_b = (2 + 3) / 2      # equals 2.5
mean_c = (4 + 5 + 6) / 3  # equals 5
average = (mean_a + mean_b + mean_c) / 3  # equals 2.667
```

As you can see, we end up with an incorrect average value! We will need to change our strategy.

---

Instead of calculating local averages, let's separate the calculation. For each player, we will create a (sum, count) tuple where the sum is the total of that player's scores and the count is number of scores encountered. This should allow us to weigh the total average according to the counts for each player and hopefully end up with the right average!

In [ ]:
# Replace the dots with the right code!

names_and_score_counts = grouped_names_and_scores.mapValues(lambda x: (..., ...))
names_and_score_counts.takeSample(withReplacement=False, num=10)

Alright, looking good so far! Let's use `map` to extract the (sum, count) tuples. Then we can sum up all the score sums and all the counts separately, and divide the total sum by the total count to end up with a result.

In [ ]:
# Replace the dots with the right code!

score_count_tuples = names_and_score_counts.map(lambda x: ...)
score_sum = score_count_tuples.map(lambda x: ...).sum()
count_sum = score_count_tuples.map(lambda x: ...).sum()
average = score_sum / count_sum

average

At last, the correct answer! Now that we have weighted the individual score sums correctly, we end up with the right average score.

---

This may all seem very complicated, and you're right to think so. With the added functionality and abstractions of Spark over classic MapReduce, we can actually calculate the average score much more easily!

In [ ]:
names_and_scores.map(lambda x: x[1]).sum() / names_and_scores.count()